In [1]:
import os
import gc
import csv
import scipy

import numerapi
import numpy as np
import pandas as pd

import lightgbm as lgb

from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform

from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

from sklearn.ensemble import RandomForestRegressor


In [2]:
TOURNAMENT_NAME = "nomi"
TARGET_NAME = f"target"
PREDICTION_NAME = f"prediction"

BENCHMARK = 0
BAND = 0.2

#-----------------------------------------------------

# Submissions are scored by spearman correlation
def score(df):
    
    
    # method="first" breaks ties based on order in array
    return np.corrcoef(
        df[TARGET_NAME],
        df[PREDICTION_NAME].rank(pct=True, method="first")
    )[0, 1]

def correlation(predictions, targets):
    ranked_preds = predictions.rank(pct=True, method="first")
    return np.corrcoef(ranked_preds, targets)[0, 1]

# The payout function
def payout(scores):
    return ((scores - BENCHMARK) / BAND).clip(lower=-1, upper=1)



def read_csv(file_path):
    """An efficient way to load csv."""
    with open(file_path, 'r') as f:
        column_names = next(csv.reader(f))
        dtypes = {x: np.float16 for x in column_names if
                  x.startswith(('feature', 'target'))}
    return pd.read_csv(file_path, dtype=dtypes)


def get_group_stats(df: pd.DataFrame) -> pd.DataFrame:
        for group in ["intelligence", "wisdom", "charisma", "dexterity", "strength", "constitution"]:
            cols = [col for col in df.columns if group in col]
            df[f"feature_{group}_mean"] = df[cols].mean(axis=1)
            df[f"feature_{group}_std"] = df[cols].std(axis=1)
            df[f"feature_{group}_skew"] = df[cols].skew(axis=1)
        return df

    
def power_vars(df: pd.DataFrame, power: int) -> pd.DataFrame:
    for col in df.columns:
        if col in feature_names:
            df[f"{col}_squared"] = df[col] ** power
        
    return df

In [3]:
napi = numerapi.NumerAPI(verbosity="info")


In [4]:
# download current dataset
napi.download_current_dataset(unzip=True)

2021-09-07 09:02:13,391 INFO numerapi.utils: starting download
./numerai_dataset_280.zip: 420MB [02:15, 3.10MB/s]                                                                                                                                                          
2021-09-07 09:04:28,939 INFO numerapi.base_api: unzipping file...


'./numerai_dataset_280.zip'

In [5]:
#getting the latest round information
current_ds = napi.get_current_round()
latest_round = os.path.join('numerai_dataset_'+str(current_ds))

In [6]:
#loading 
print("# Loading data...")
# The training data is used to train your model how to predict the targets.
training_data = read_csv(os.path.join(latest_round, "numerai_training_data.csv")).set_index("id")
# The tournament data is the data that Numerai uses to evaluate your model.
tournament_data = read_csv(os.path.join(latest_round, "numerai_tournament_data.csv")).set_index("id")

# example_preds = read_csv(os.path.join(latest_round, "example_predictions_target_kazutsugi.csv")
validation_data = tournament_data[tournament_data.data_type == "validation"]

# Loading data...


In [7]:
feature_names = [f for f in training_data.columns if f.startswith("feature")]
print(f"Loaded {len(feature_names)} features")
cols = feature_names+[TARGET_NAME]

Loaded 310 features


In [8]:
training_data = get_group_stats(training_data)
tournament_data = get_group_stats(tournament_data)
validation_data = get_group_stats(validation_data)

/var/folders/cv/ns61j9ns4b17x67b75m0yv4jlfq4r6/T/ipykernel_38553/1097664127.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"feature_{group}_mean"] = df[cols].mean(axis=1)
/var/folders/cv/ns61j9ns4b17x67b75m0yv4jlfq4r6/T/ipykernel_38553/1097664127.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"feature_{group}_std"] = df[cols].std(axis=1)
/var/folders/cv/ns61j9ns4b17x67b75m0yv4jlfq4r6/T/ipykernel_38553/1097664127.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

In [9]:
training_data = power_vars(training_data, 2)
tournament_data = power_vars(tournament_data, 2)
validation_data = power_vars(validation_data, 2)

/var/folders/cv/ns61j9ns4b17x67b75m0yv4jlfq4r6/T/ipykernel_38553/1097664127.py:51: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{col}_squared"] = df[col] ** power
/var/folders/cv/ns61j9ns4b17x67b75m0yv4jlfq4r6/T/ipykernel_38553/1097664127.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"{col}_squared"] = df[col] ** power


In [10]:
training_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 501808 entries, n000315175b67977 to nffba5460b572cfa
Columns: 641 entries, era to feature_wisdom46_squared
dtypes: float16(639), object(2)
memory usage: 623.1+ MB


In [11]:
# A parameter grid for XGBoost
#params = {
#    'n_estimators':[1000],
#    'min_child_weight':[4,5], 
#    'gamma':[i/10.0 for i in range(3,6)],  
#    'subsample':[i/10.0 for i in range(6,11)],
#    'colsample_bytree':[i/10.0 for i in range(6,11)], 
#    'max_depth': [3,6,9,12],
#    'objective': ['reg:squarederror'],
#    'booster': ['gblinear'],
#    'eval_metric': ['rmse'],
#    'learning_rate': list(np.linspace(0.005, .3, num=8)),
#}

params = {
    'iterations':[5000],
    'depth':sp_randint(3,15), 
    'learning_rate': sp_uniform(0.005, 0.3),
    'reg_lambda': sp_randint(1, 5),
    'use_best_model':[True],
    'min_data_in_leaf': sp_randint(1, 5)
}

fit_params={"early_stopping_rounds":15, 
            "eval_set" :[(validation_data.drop(['era','data_type','target'], axis=1).astype(np.float32), validation_data[TARGET_NAME].astype(np.float32))]}

reg = CatBoostRegressor()

# run randomized search
n_iter_search = 60
random_search = RandomizedSearchCV(reg, 
                                   param_distributions=params,
                                   n_iter=n_iter_search, 
                                   cv=3, 
                                   scoring='neg_mean_squared_error',
                                   verbose=2)

random_search.fit(training_data.drop(['era','data_type','target'], axis=1).astype(np.float32), training_data[TARGET_NAME].astype(np.float32), **fit_params)

Fitting 3 folds for each of 60 candidates, totalling 180 fits
0:	learn: 0.2232460	test: 0.2234907	best: 0.2234907 (0)	total: 336ms	remaining: 27m 57s
1:	learn: 0.2232285	test: 0.2234835	best: 0.2234835 (1)	total: 723ms	remaining: 30m 7s
2:	learn: 0.2232039	test: 0.2234797	best: 0.2234797 (2)	total: 1.04s	remaining: 28m 51s
3:	learn: 0.2231937	test: 0.2234844	best: 0.2234797 (2)	total: 1.49s	remaining: 30m 58s
4:	learn: 0.2231815	test: 0.2234771	best: 0.2234771 (4)	total: 1.87s	remaining: 31m 12s
5:	learn: 0.2231710	test: 0.2234796	best: 0.2234771 (4)	total: 2.19s	remaining: 30m 21s
6:	learn: 0.2231574	test: 0.2234764	best: 0.2234764 (6)	total: 2.47s	remaining: 29m 24s
7:	learn: 0.2231451	test: 0.2234710	best: 0.2234710 (7)	total: 2.83s	remaining: 29m 25s
8:	learn: 0.2231321	test: 0.2234690	best: 0.2234690 (8)	total: 3.15s	remaining: 29m 7s
9:	learn: 0.2231223	test: 0.2234638	best: 0.2234638 (9)	total: 3.47s	remaining: 28m 49s
10:	learn: 0.2231132	test: 0.2234659	best: 0.2234638 (9)	tot

4:	learn: 0.2228741	test: 0.2235009	best: 0.2234856 (0)	total: 3.14s	remaining: 52m 18s
5:	learn: 0.2228113	test: 0.2235075	best: 0.2234856 (0)	total: 3.74s	remaining: 51m 53s
6:	learn: 0.2227355	test: 0.2235203	best: 0.2234856 (0)	total: 4.34s	remaining: 51m 35s
7:	learn: 0.2226699	test: 0.2235134	best: 0.2234856 (0)	total: 4.94s	remaining: 51m 21s
8:	learn: 0.2226101	test: 0.2235134	best: 0.2234856 (0)	total: 5.49s	remaining: 50m 47s
9:	learn: 0.2225400	test: 0.2235226	best: 0.2234856 (0)	total: 6.09s	remaining: 50m 38s
10:	learn: 0.2224874	test: 0.2235330	best: 0.2234856 (0)	total: 6.76s	remaining: 51m 7s
11:	learn: 0.2224190	test: 0.2235449	best: 0.2234856 (0)	total: 7.36s	remaining: 50m 58s
12:	learn: 0.2223594	test: 0.2235632	best: 0.2234856 (0)	total: 8.03s	remaining: 51m 21s
13:	learn: 0.2223094	test: 0.2235738	best: 0.2234856 (0)	total: 9.24s	remaining: 54m 50s
14:	learn: 0.2222433	test: 0.2235765	best: 0.2234856 (0)	total: 10s	remaining: 55m 34s
15:	learn: 0.2221833	test: 0.2

17:	learn: 0.2224728	test: 0.2235241	best: 0.2234749 (3)	total: 11.2s	remaining: 51m 40s
18:	learn: 0.2224311	test: 0.2235205	best: 0.2234749 (3)	total: 11.7s	remaining: 51m 19s
Stopped by overfitting detector  (15 iterations wait)

bestTest = 0.2234749423
bestIteration = 3

Shrink model to first 4 iterations.
[CV] END depth=8, iterations=5000, learning_rate=0.140127557400898, min_data_in_leaf=2, reg_lambda=2, use_best_model=True; total time=  17.0s
0:	learn: 0.2231926	test: 0.2234931	best: 0.2234931 (0)	total: 792ms	remaining: 1h 5m 57s
1:	learn: 0.2231317	test: 0.2234871	best: 0.2234871 (1)	total: 1.79s	remaining: 1h 14m 29s
2:	learn: 0.2230834	test: 0.2234881	best: 0.2234871 (1)	total: 2.43s	remaining: 1h 7m 22s
3:	learn: 0.2230306	test: 0.2234893	best: 0.2234871 (1)	total: 3.02s	remaining: 1h 2m 48s
4:	learn: 0.2229803	test: 0.2234841	best: 0.2234841 (4)	total: 3.59s	remaining: 59m 51s
5:	learn: 0.2229290	test: 0.2234863	best: 0.2234841 (4)	total: 4.27s	remaining: 59m 13s
6:	learn:

64:	learn: 0.2229527	test: 0.2234623	best: 0.2234571 (53)	total: 24.5s	remaining: 31m 3s
65:	learn: 0.2229495	test: 0.2234621	best: 0.2234571 (53)	total: 24.9s	remaining: 31m 3s
66:	learn: 0.2229457	test: 0.2234626	best: 0.2234571 (53)	total: 25.3s	remaining: 31m 1s
67:	learn: 0.2229439	test: 0.2234625	best: 0.2234571 (53)	total: 25.7s	remaining: 31m 3s
68:	learn: 0.2229399	test: 0.2234634	best: 0.2234571 (53)	total: 26s	remaining: 31m 1s
Stopped by overfitting detector  (15 iterations wait)

bestTest = 0.2234570503
bestIteration = 53

Shrink model to first 54 iterations.
[CV] END depth=4, iterations=5000, learning_rate=0.0847370911643415, min_data_in_leaf=4, reg_lambda=3, use_best_model=True; total time=  31.5s
0:	learn: 0.2232741	test: 0.2234890	best: 0.2234890 (0)	total: 379ms	remaining: 31m 33s
1:	learn: 0.2232639	test: 0.2234859	best: 0.2234859 (1)	total: 823ms	remaining: 34m 15s
2:	learn: 0.2232512	test: 0.2234835	best: 0.2234835 (2)	total: 1.21s	remaining: 33m 35s
3:	learn: 0.22

46:	learn: 0.2229683	test: 0.2234497	best: 0.2234497 (46)	total: 18.9s	remaining: 33m 14s
47:	learn: 0.2229647	test: 0.2234494	best: 0.2234494 (47)	total: 19.3s	remaining: 33m 13s
48:	learn: 0.2229614	test: 0.2234493	best: 0.2234493 (48)	total: 19.7s	remaining: 33m 6s
49:	learn: 0.2229585	test: 0.2234493	best: 0.2234493 (49)	total: 20s	remaining: 33m
50:	learn: 0.2229549	test: 0.2234474	best: 0.2234474 (50)	total: 20.4s	remaining: 32m 55s
51:	learn: 0.2229491	test: 0.2234475	best: 0.2234474 (50)	total: 20.7s	remaining: 32m 49s
52:	learn: 0.2229472	test: 0.2234472	best: 0.2234472 (52)	total: 21.1s	remaining: 32m 49s
53:	learn: 0.2229440	test: 0.2234490	best: 0.2234472 (52)	total: 21.5s	remaining: 32m 48s
54:	learn: 0.2229411	test: 0.2234489	best: 0.2234472 (52)	total: 21.8s	remaining: 32m 44s
55:	learn: 0.2229382	test: 0.2234498	best: 0.2234472 (52)	total: 22.2s	remaining: 32m 42s
56:	learn: 0.2229355	test: 0.2234509	best: 0.2234472 (52)	total: 22.7s	remaining: 32m 47s
57:	learn: 0.2229

...

45:	learn: 0.2222268	test: 0.2234590	best: 0.2234584 (43)	total: 4m 3s	remaining: 7h 17m 33s
46:	learn: 0.2222069	test: 0.2234600	best: 0.2234584 (43)	total: 4m 10s	remaining: 7h 19m 57s
47:	learn: 0.2221897	test: 0.2234591	best: 0.2234584 (43)	total: 4m 16s	remaining: 7h 20m 53s
48:	learn: 0.2221701	test: 0.2234593	best: 0.2234584 (43)	total: 4m 22s	remaining: 7h 21m 15s
49:	learn: 0.2221485	test: 0.2234589	best: 0.2234584 (43)	total: 4m 27s	remaining: 7h 21m 26s
50:	learn: 0.2221269	test: 0.2234575	best: 0.2234575 (50)	total: 4m 33s	remaining: 7h 21m 38s
51:	learn: 0.2221044	test: 0.2234570	best: 0.2234570 (51)	total: 4m 37s	remaining: 7h 20m 27s
52:	learn: 0.2220819	test: 0.2234578	best: 0.2234570 (51)	total: 4m 42s	remaining: 7h 19m 30s
53:	learn: 0.2220643	test: 0.2234574	best: 0.2234570 (51)	total: 4m 47s	remaining: 7h 19m 8s
54:	learn: 0.2220418	test: 0.2234569	best: 0.2234569 (54)	total: 4m 52s	remaining: 7h 18m 29s
55:	learn: 0.2220195	test: 0.2234575	best: 0.2234569 (54)	tota

24:	learn: 0.2227114	test: 0.2234612	best: 0.2234612 (24)	total: 1m 13s	remaining: 4h 4m 13s
25:	learn: 0.2226884	test: 0.2234618	best: 0.2234612 (24)	total: 1m 16s	remaining: 4h 3m 23s
26:	learn: 0.2226694	test: 0.2234619	best: 0.2234612 (24)	total: 1m 19s	remaining: 4h 2m 50s
27:	learn: 0.2226453	test: 0.2234617	best: 0.2234612 (24)	total: 1m 22s	remaining: 4h 3m 5s
28:	learn: 0.2226241	test: 0.2234610	best: 0.2234610 (28)	total: 1m 24s	remaining: 4h 2m 14s
29:	learn: 0.2226056	test: 0.2234617	best: 0.2234610 (28)	total: 1m 27s	remaining: 4h 1m 14s
30:	learn: 0.2225831	test: 0.2234616	best: 0.2234610 (28)	total: 1m 29s	remaining: 4h 21s
31:	learn: 0.2225594	test: 0.2234619	best: 0.2234610 (28)	total: 1m 32s	remaining: 3h 59m 46s
32:	learn: 0.2225356	test: 0.2234610	best: 0.2234610 (32)	total: 1m 35s	remaining: 3h 59m 24s
33:	learn: 0.2225158	test: 0.2234609	best: 0.2234609 (33)	total: 1m 38s	remaining: 3h 58m 48s
34:	learn: 0.2224924	test: 0.2234611	best: 0.2234609 (33)	total: 1m 40s

45:	learn: 0.2222256	test: 0.2234585	best: 0.2234576 (43)	total: 1m 56s	remaining: 3h 28m 46s
46:	learn: 0.2222076	test: 0.2234578	best: 0.2234576 (43)	total: 1m 58s	remaining: 3h 28m 37s
47:	learn: 0.2221903	test: 0.2234584	best: 0.2234576 (43)	total: 2m 1s	remaining: 3h 28m 34s
48:	learn: 0.2221695	test: 0.2234573	best: 0.2234573 (48)	total: 2m 3s	remaining: 3h 28m 31s
49:	learn: 0.2221494	test: 0.2234579	best: 0.2234573 (48)	total: 2m 6s	remaining: 3h 28m 34s
50:	learn: 0.2221279	test: 0.2234579	best: 0.2234573 (48)	total: 2m 9s	remaining: 3h 28m 38s
51:	learn: 0.2221053	test: 0.2234581	best: 0.2234573 (48)	total: 2m 11s	remaining: 3h 28m 28s
52:	learn: 0.2220844	test: 0.2234570	best: 0.2234570 (52)	total: 2m 13s	remaining: 3h 28m 20s
53:	learn: 0.2220668	test: 0.2234563	best: 0.2234563 (53)	total: 2m 16s	remaining: 3h 28m 9s
54:	learn: 0.2220446	test: 0.2234548	best: 0.2234548 (54)	total: 2m 18s	remaining: 3h 28m 5s
55:	learn: 0.2220203	test: 0.2234544	best: 0.2234544 (55)	total: 2

61:	learn: 0.2229859	test: 0.2234440	best: 0.2234398 (52)	total: 40.6s	remaining: 53m 52s
62:	learn: 0.2229839	test: 0.2234444	best: 0.2234398 (52)	total: 41.4s	remaining: 54m
63:	learn: 0.2229810	test: 0.2234438	best: 0.2234398 (52)	total: 42s	remaining: 54m 1s
64:	learn: 0.2229794	test: 0.2234432	best: 0.2234398 (52)	total: 42.7s	remaining: 54m 3s
65:	learn: 0.2229771	test: 0.2234439	best: 0.2234398 (52)	total: 43.3s	remaining: 54m
66:	learn: 0.2229749	test: 0.2234452	best: 0.2234398 (52)	total: 44s	remaining: 53m 56s
67:	learn: 0.2229718	test: 0.2234452	best: 0.2234398 (52)	total: 44.6s	remaining: 53m 52s
Stopped by overfitting detector  (15 iterations wait)

bestTest = 0.2234397785
bestIteration = 52

Shrink model to first 53 iterations.


RandomizedSearchCV(cv=3,
                   estimator=<catboost.core.CatBoostRegressor object at 0x7ff8576f4340>,
                   n_iter=60,
                   param_distributions={'depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ff8b24f5cd0>,
                                        'iterations': [5000],
                                        'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ff86621eeb0>,
                                        'min_data_in_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ff8662e3370>,
                                        'reg_lambda': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ff8662e3130>,
                                        'use_best_model': [True]},
                   scoring='neg_mean_squared_error', verbose=2)

In [12]:
print("Generating predictions on training data...")
training_preds = random_search.predict(training_data.drop(['era','data_type','target'], axis=1).astype(np.float32).astype(np.float32))
training_data[PREDICTION_NAME] = training_preds
gc.collect()

print("Generating predictions on tournament data...")
tournament_preds = random_search.predict(tournament_data.drop(['era','data_type','target'], axis=1).astype(np.float32).astype(np.float32))
tournament_data[PREDICTION_NAME] = tournament_preds

# Check the per-era correlations on the training set (in sample)
train_correlations = training_data.groupby("era").apply(score)
print(f"On training the correlation has mean {train_correlations.mean()} and std {train_correlations.std()}")
print(f"On training the average per-era payout is {payout(train_correlations).mean()}")

# Check the per-era correlations on the validation set (out of sample)
validation_data = tournament_data[tournament_data.data_type == "validation"]
validation_correlations = validation_data.groupby("era").apply(score)
print(f"On validation the correlation has mean {validation_correlations.mean()} and "
        f"std {validation_correlations.std()}")
print(f"On validation the average per-era payout is {payout(validation_correlations).mean()}")

#FEAT_EXPOSURE:
corr_list = []
for feature in feature_names:
    corr_list.append(np.corrcoef(tournament_data[feature].values, tournament_data[PREDICTION_NAME])[0,1])
corr_series = pd.Series(corr_list, index=feature_names)
print("Feat. exposure: ", corr_series.describe()['std'])

Generating predictions on training data...
Generating predictions on tournament data...
On training the correlation has mean 0.05831450171574314 and std 0.03461791913153154
On training the average per-era payout is 0.2915725085787157
On validation the correlation has mean 0.0201183168134707 and std 0.03226390033276007
On validation the average per-era payout is 0.1005915840673535
Feat. exposure:  0.09989891925305641


In [ ]:
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 10)]
max_features = ['log2', 'sqrt']
max_depth = [int(x) for x in np.linspace(start = 1, stop = 15, num = 15)]
min_samples_split = [int(x) for x in np.linspace(start = 2, stop = 50, num = 10)]
min_samples_leaf = [int(x) for x in np.linspace(start = 2, stop = 50, num = 10)]
bootstrap = [True, False]
param_dist = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

rfc = RandomForestRegressor()

rs = RandomizedSearchCV(rfc, 
                        param_dist, 
                        n_iter = 100, 
                        cv = 3, 
                        verbose = 1, 
                        random_state=314)
rs.fit(training_data.drop(['era','data_type','target'], axis=1), training_data[TARGET_NAME])
rs.best_params_


In [ ]:
print("Generating predictions on training data...")
training_preds = random_search.predict(training_data[feature_names])
training_data[PREDICTION_NAME] = training_preds
gc.collect()

print("Generating predictions on tournament data...")
tournament_preds = random_search.predict(tournament_data[feature_names])
tournament_data[PREDICTION_NAME] = tournament_preds

# Check the per-era correlations on the training set (in sample)
train_correlations = training_data.groupby("era").apply(score)
print(f"On training the correlation has mean {train_correlations.mean()} and std {train_correlations.std()}")
print(f"On training the average per-era payout is {payout(train_correlations).mean()}")

# Check the per-era correlations on the validation set (out of sample)
validation_data = tournament_data[tournament_data.data_type == "validation"]
validation_correlations = validation_data.groupby("era").apply(score)
print(f"On validation the correlation has mean {validation_correlations.mean()} and "
        f"std {validation_correlations.std()}")
print(f"On validation the average per-era payout is {payout(validation_correlations).mean()}")

#FEAT_EXPOSURE:
corr_list = []
for feature in feature_names:
    corr_list.append(np.corrcoef(tournament_data[feature].values, tournament_data[PREDICTION_NAME])[0,1])
corr_series = pd.Series(corr_list, index=feature_names)
print("Feat. exposure: ", corr_series.describe()['std'])

In [ ]:
random_search.best_estimator_

In [ ]:
fit_params={"early_stopping_rounds":30, 
            "eval_metric" : 'logloss', 
            "eval_set" : [(validation_data[feature_names],validation_data[TARGET_NAME])],
            'eval_names': ['valid'],
            'verbose': 100,
            'categorical_feature': 'auto'}

In [ ]:
param_test ={'num_leaves': sp_randint(6, 50), 
             'min_child_samples': sp_randint(100, 500), 
             'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
             'subsample': sp_uniform(loc=0.2, scale=0.8), 
             'colsample_bytree': sp_uniform(loc=0.4, scale=0.6),
             'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
             'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100]}

In [ ]:
#This parameter defines the number of HP points to be tested
n_HP_points_to_test = 100


#n_estimators is set to a "large value". The actual number of trees build will depend on early stopping and 5000 define only the absolute maximum
clf = lgb.LGBMRegressor(max_depth=-1, random_state=314, silent=True, metric='None', n_jobs=4, n_estimators=5000)
gs = RandomizedSearchCV(
    estimator=clf, 
    param_distributions=param_test, 
    n_iter=n_HP_points_to_test,
    scoring='logloss',
    cv=3,
    refit=True,
    random_state=314,
    verbose=True)

In [ ]:
gs.fit(training_data[feature_names], training_data[TARGET_NAME], **fit_params)
print('Best score reached: {} with params: {} '.format(gs.best_score_, gs.best_params_))

In [ ]:
param_test ={'num_leaves': sp_randint(6, 50), 
             'min_child_samples': sp_randint(100, 500), 
             'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
             'subsample': sp_uniform(loc=0.2, scale=0.8), 
             'colsample_bytree': sp_uniform(loc=0.4, scale=0.6),
             'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
             'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100]}


lgb_estimator= lgb.LGBMRegressor(max_depth=-1, random_state=314, metric='None', n_estimators=5000, early_stopping_rounds=30)
g_lgbm = RandomizedSearchCV(estimator=lgb_estimator, param_distributions=param_test, n_iter=60 ,n_jobs = 2, cv= 3, verbose=10)
lgb_model = g_lgbm.fit(X=training_data[feature_names], y=training_data[TARGET_NAME], eval_set = (validation_data[feature_names],validation_data[TARGET_NAME]), eval_metric='r2')